In [1]:
# Imports

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from numpy.matlib import repmat

In [17]:
class FunctionCompilation():
    # feval equivalent
    def callFunction(self, name, val):
        fn = getattr(self, name, None)
        if fn is not None:
            return fn(val)
        else:
            print('Undefined function call')
            return
        
        
    def ackley(self, val):
        dim = val.shape[1]
        
        f1 = sum([val[:, :1+dim]**2])
        f2 = sum([np.cos(2*np.pi * val[:, :1+dim])])
        
        return -20 * np.exp(-0.2 * np.sqrt(1/dim * f1)) - np.exp(1/dim*f2) + a + np.exp(1)

In [20]:
def HyDE_DF(deParameters, otherParameters, initialSolution):
    
    # Generate population
    def genpop(a, b, lowMatrix, upMatrix):
        return np.random.uniform(low=lowMatrix, high=upMatrix, size=(a, b))
    
    
    # Trial generation
    def generate_trial(F_weight, F_CR, FM_pop, FVr_bestmemit, I_NP, I_D, FVr_rot, other):
    
        # Save the old population
        FM_popold = FM_pop

        # Index pointer array
        FVr_ind = np.random.Generator.permutation(5)

        # Shuffle locations of vectors
        FVr_a1 = np.random.Generator.permutation(I_NP + 1)

        # Rotate indices by ind[0] positions
        FVr_rt = (FVr_rot + FVr_ind[0]) % I_NP

        # Rotate vector locations
        FVr_a2 = FVr_a1[FVr_rt + 1]

        # Shuffled population 1
        FM_pm1 = FM_popold[FVr_a1, :]

        # Shuffled population 2
        FM_pm2 = FM_popold[FVr_a2, :]


        # Meaning the same F_CR for all individuals
        if len(F_CR) == 1:

            # All random numbers < F_CR are 1, 0 otherwise
            FM_mui = int(np.random.normal(shape=(I_NP, I_D)) < F_CR)

            # Inverse mask to FM_mui
            FM_mpo = int(FM_mui < 0.5)

        # Meaning a different F_CR for each individual
        else:
            # All random numbers < F_CR are 1, 0 otherwise
            FM_mui = int(np.random.normal(shape=(I_NP, I_D)) < repmat(F_CR, 1, I_D))

            # Inverse mask to FM_mui
            FM_mpo = FM_mui < 0.5
            
            
    # Update aux function        
    def _update(p, lowMatrix, upMatrix, BRM, FM_base):
        if BRM == 1: # Our method
            # [popsize, dim] = size(p)
            idx = np.where(p < lowMatrix)
            p[idx] = lowMatrix[idx]

            idx = np.where(p > upMatrix)
            p[idx] = upMatrix[idx]
        elif BRM == 2: # Random initialization
            idx = [np.where(p < lowMatrix), np.where(p > upMatrix)]
            replace = np.random.uniform(low=lowMatrix[idx], high=upMatrix[idx], size=(len(idx), 1))
            p[idx] = replace
        elif BRM == 3: # Bounce-back
            idx = np.where(p < lowMatrix)
            p[idx] = np.random.uniform(low=lowMatrix[idx], high=FM_base[idx], size=(len(idx), 1))
            idx = np.where(p > upMatrix)
            p[idx] = np.random.uniform(low=FM_base[idx], high=upMatrix[idx], size=(len(idx), 1))

        return


        #%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% 
        FM_bm = repmat(FVr_bestmemit, I_NP, 1)

        # Linear decrease
        a = other.a 

        # Exponential decreasing function
        ginv = np.exp(1-(1/a**2)) 


        #differential variation
        diff_var = ginv * (repmat(F_weight[:, 0], 1, I_D) * (FM_bm * (repmat(F_weight[:, 1], 1, I_D) + np.random.normal(shape=(I_NP, I_D)) - FM_popold)))

        FM_ui = FM_popold + repmat(F_weight[:, 2], 1, I_D) * (FM_pm1 - FM_pm2) + diff_var

        FM_ui = FM_popold * FM_mpo + FM_ui * FM_mui
        FM_base = FM_bm
        print('HyDE-DF')

        return FM_ui, FM_base, msg
    
    
    
    #-----This is just for notational convenience and to keep the code uncluttered.--------
    I_NP = deParameters.I_NP
    F_weight = deParameters.F_weight
    F_CR = deParameters.F_CR
    I_D = otherParameters.dim     #Number of variables or dimension
    deParameters.nVariables = I_D
    FVr_minbound = otherParameters.lowerlimit * np.array(np.ones((1,otherParameters.dim)))
    FVr_maxbound = otherParameters.upperlimit * np.array(np.ones((1,otherParameters.dim)))
    I_itermax = deParameters.I_itermax
    
    
    #Repair boundary method employed
    BRM = deParameters.I_bnd_constr   #1: bring the value to bound violated
                                      #2: repair in the allowed range
                                      #3: Bounce-back
            
    # Get Objective Function
    fnc= otherParameters.objfun
    
    # Function caller initialization
    fn_obj = FunctionCompilation()
    
    #-----Check input variables---------------------------------------------
    if I_NP < 5:
        I_NP = 5
        print('I_NP increased to minimal value 5\n')
    
    if (F_CR < 0) | (F_CR > 1):
        F_CR = 0.5
        print('F_CR should be from interval [0, 1] - set to default value 0.5\n')
        
    if I_itermax <= 0:
        I_itermax = 200
        print('I_itermax should be > 0 - set to default value 200\n')
    
    
    #-----Initialize population and some arrays-------------------------------
    #%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
    # pre-allocation of loop variables
    fitMaxVector = np.empty((1, I_itermax))
    fitMaxVector[:] = np.nan
    
    
    # limit iterations by threshold
    gen = 1; #iterations
    
    
    #%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
    #----FM_pop is a matrix of size I_NPx(I_D+1). It will be initialized------
    #----with random values between the min and max values of the-------------
    #----parameters-----------------------------------------------------------
    # FLC modification - vectorization
    minPositionsMatrix = repmat(FVr_minbound, I_NP, 1)
    maxPositionsMatrix = repmat(FVr_maxbound, I_NP, 1)
    deParameters.minPositionsMatrix = minPositionsMatrix
    deParameters.maxPositionsMatrix = maxPositionsMatrix
    
    # generate initial population.
    FM_pop = genpop(I_NP,I_D,minPositionsMatrix,maxPositionsMatrix)
    
    #If you want to inject initial solutions
    if nargin > 2: 
        noInitialSolutions = initialSolution.shape[0]
        FM_pop[1:noInitialSolutions, :] = initialSolution
        
        
    #%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
    #------Evaluate the best member after initialization----------------------
    # Modified by FLC
    S_val = fn_obj.callFunction(fnc, FM_pop)
    I_best_index = np.argmin(S_val); # This mean that the best individual correspond to the best worst performance
    FVr_bestmemit = FM_pop[I_best_index, :] # best member of current iteration
    
    fitMaxVector[:, 0] = S_val(I_best_index) #We save the mean value and mean penalty value
    # The user can decide to save the mean, best, or any other value here
    
    
    #------DE-Minimization------------------------------------------------
    #------FM_popold is the population which has to compete. It is--------
    #------static through one iteration. FM_pop is the newly--------------
    #------emerging population.-------------------------------------------
    FVr_rot  = np.arange(0, I_NP, 1)             # rotating index array (size I_NP)
    
    
    
    #%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
    #% HYDE self-adaptive parameters
    F_weight_old = repmat(F_weight, I_NP, 3)
    F_weight = F_weight_old
    F_CR_old = repmat(F_CR, I_NP, 1)
    F_CR = F_CR_old

    
    while gen < I_itermax:  #%&&  fitIterationGap >= threshold
        #% Calculate decay function factor a = itr / MaxItr; 
        other.a = (I_itermax-gen) / I_itermax


        #% Update HyDE-DF values              
        value_R = np.random.normal(size=(I_NP,3))
        ind1 = value_R < 0.1
        ind2 = np.random.normal(size=(I_NP,1)) < 0.1    
        
        F_weight[:] = F_weight_old[:]
        F_weight[ind1] = 0.1 + np.random.normal(size=(sum(sum(ind1)), 1)) * 0.9
        
        F_CR[:] = F_CR_old[:]
        F_CR[ind2] = np.random.normal(size=(sum(ind2), 1))
        
        #%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
        FM_ui, FM_base, _ = generate_trial(F_weight, F_CR, FM_pop, FVr_bestmemit,I_NP, I_D, FVr_rot,other)
        #%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
 
        ## Boundary Control
        FM_ui = update(FM_ui,minPositionsMatrix,maxPositionsMatrix,BRM,FM_base)
        
        
        ## Evaluation of new Pop
        S_val_temp = fn_obj.callFunction(fnc, FM_ui)
        
        
        ## Elitist Selection
        ind = np.where(S_val_temp < S_val)
        S_val[ind] = S_val_temp[ind]
        FM_pop[ind, :] = FM_ui[ind, :]
        
        
        ## Update best results
        I_best_index = np.argmin(S_val)
        FVr_bestmemit = FM_pop[I_best_index, :]
        
        
        ## Save best parameters (similar to jDE)
        F_weight_old[ind, :] = F_weight[ind, :]
        F_CR_old[ind] = F_CR[ind]
        
        
        ## Store fitness evolution and obj fun evolution as well
        fitMaxVector[:, gen] = S_bestval
        
        
    Fit_and_p = fitMaxVector[0, gen]
    
    return Fit_and_p, FVr_bestmemit, fitMaxVector